In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

%matplotlib inline

In [2]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteTelemetry(ATDome, 0, position) falling behind; read 33 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages


[[None, None, None, None, None, None, None], None]

RemoteEvent(ATAOS, 0, detailedState) falling behind; read 11 messages


In [5]:
# Declare target name
target_name='iotCen'

In [42]:
focus_offset=0.011408860000000003-0.00524873+0.04107877-0.03388079
x_offset=-0.51889512+0.73684108-0.1408324-0.44613063
y_offset=-0.08800368999999994+0.30172143+0.012768-0.15228448
expTime=40.

In [43]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [44]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
dz=0.8
1580016146-iotCen_intra-0-1
1580016193-iotCen_extra-0-1
['1580016146-iotCen_intra-0-1.fits'], ['1580016193-iotCen_extra-0-1.fits']
144.96369175186248


In [45]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

/home/saluser/develop/cwfs/python/data/1580016146-iotCen_intra-0-1.fits /home/saluser/develop/cwfs/python/data/1580016193-iotCen_extra-0-1.fits


RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 68 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 73 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages


In [19]:
await cam.cmd_stopLiveView.start()

In [20]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=30., shutter=True, imageSequenceName=target_name+'_focus')

In [26]:
in_foc = await cam.evt_endReadout.next(flush=False, timeout=5)

In [22]:
in_foc.imageName

'1580014873-iotCen_focus-0-1'

In [24]:
await cam.cmd_setValue.set_start(parametersAndValues="filter,2")

In [25]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=30., shutter=True, imageSequenceName=target_name+'_foc_filter2')

In [28]:
in_foc = await cam.evt_endReadout.next(flush=False, timeout=5)
print(f"{in_foc.imageName}.fits")

1580015089-iotCen_foc_filter2-0-1.fits


In [29]:
await cam.cmd_setValue.set_start(parametersAndValues="filter,3")

In [30]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=30., shutter=True, imageSequenceName=target_name+'_foc_filter2')
in_foc = await cam.evt_endReadout.next(flush=False, timeout=5)
print(f"{in_foc.imageName}.fits")

1580015246-iotCen_foc_filter2-0-1.fits


In [31]:
await cam.cmd_setValue.set_start(parametersAndValues="filter,4")

In [32]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=30., shutter=True, imageSequenceName=target_name+'_foc_filter4')
in_foc = await cam.evt_endReadout.next(flush=False, timeout=5)
print(f"{in_foc.imageName}.fits")

1580015365-iotCen_foc_filter4-0-1.fits


In [33]:
await cam.cmd_setValue.set_start(parametersAndValues="filter,5")

In [34]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=30., shutter=True, imageSequenceName=target_name+'_foc_filter5')
in_foc = await cam.evt_endReadout.next(flush=False, timeout=5)
print(f"{in_foc.imageName}.fits")

1580015463-iotCen_foc_filter5-0-1.fits


In [35]:
await cam.cmd_setValue.set_start(parametersAndValues="filter,6")

In [36]:
await cam.cmd_takeImages.set_start(numImages=1, expTime=30., shutter=True, imageSequenceName=target_name+'_foc_filter6')
in_foc = await cam.evt_endReadout.next(flush=False, timeout=5)
print(f"{in_foc.imageName}.fits")

1580015494-iotCen_foc_filter6-0-1.fits


In [37]:
await cam.cmd_setValue.set_start(parametersAndValues="filter,1")

In [38]:
await cam.cmd_startLiveView.set_start(expTime=0.5)